In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 한국어 training

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("jojo0217/korean_safe_conversation", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/26979 [00:00<?, ? examples/s]

Map:   0%|          | 0/26979 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/26979 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 26,979 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.153600
2,2.278700
3,2.353000
4,2.252000
5,2.289200
6,2.286700
7,2.002100
8,1.772900
9,1.651400
10,1.726200


# Train data EDA 및 전처리

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import ast

train_data = "/content/train_data.csv"
df = pd.read_csv(train_data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문제      9000 non-null   object
 1   선택지     9000 non-null   object
 2   답안      9000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 211.1+ KB


In [ ]:
# '선택지' 열을 리스트로 변환
df['선택지'] = df['선택지'].apply(ast.literal_eval)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)

In [ ]:
for i, val in enumerate(df['선택지']):
    try:
        ast.literal_eval(val)
    except (ValueError, SyntaxError):
        print(f"문제가 발생한 행 인덱스: {i}, 값: {val}")


문제가 발생한 행 인덱스: 2943, 값: ['국가가 안정성을 제공하지 못함으로써 사회의 각 구성 요소가 자신의 안녕을 유지하기 위해 경쟁하게 되어 불안을 초래하는 상황을 말합니다. 이 조건은 자가 증식적이며, 체제를 확보하기 위한 조치가 더 큰 저항을 불러일으키기 때문에 반영구적인 긴급 무정부 상태가 됩니다.', '약소국의 불안정 딜레마는 주로 외부 조건으로 인해 발생하며, 구조적 무정부 상태와 유사한 상황을 초래합니다. 약소국이 자국의 지역적 입지를 개선하기 위한 조치를 취할 때 지역 내 불안을 조성합니다.', '약소국의 불안정 딜레마는 사회의 각 구성 요소가 자신의 안녕과 이익을 보호하고 유지하기 위한 경쟁에서 비롯됩니다. 그러나 지배 엘리트는 사회적 경쟁 영역과 분리되어 정책 딜레마를 초래합니다. 질서를 회복하기 위해 폭력 수단을 사용할 경우 체제의 기반이 약화됩니다.', '약소국의 불안정 딜레마는 정치적 및 제도적 중심화와 힘의 독점이 부족한 상황에서 발생합니다. 그러나 제도를 강화하기 위해 무력을 동원하면 이 과정을 중단시킬 수 있습니다. '국가성'을 육성하지 못하는 것은 폭력 사용으로 반전됩니다. 사회적 불안은 긴급하지만 개발되지 않은 무정부 상태의 반영구적 상황입니다.']
문제가 발생한 행 인덱스: 3257, 값: ['모든 RNA가 공통된 3' 말단에서 끝나고 중첩된 세트 전사체를 생성합니다', '긴 RNA 유전체와의 재조합을 이용합니다', '변이율이 높지 않습니다', '캡이 씌워진 세포 mRNA를 사용합니다']
문제가 발생한 행 인덱스: 4411, 값: ['정치적 편견 없이 진실되고 정확한 설명이어야 한다', '생존하지 못한 유사한 모든 문서를 대표해야 한다', '문자 그대로의 의미와 해석 가능한 의미를 모두 가져야 한다', '알려진 저자의 '신뢰할 수 있는' 원본 또는 신뢰할 수 있는 사본이어야 한다']
문제가 발생한 행 인덱스: 5439, 값: ['가난한 사람들이 상대적으로 박탈감을 느끼게 하는 문화적으로 가치 있는 상품과 생활 수준', '

In [ ]:
df = df.drop([2943, 3257, 4411, 5439, 7115, 8856])  # 문제가 발생한 행을 삭제

In [ ]:
# '선택지' 열을 리스트로 변환
df['선택지'] = df['선택지'].apply(ast.literal_eval)

In [ ]:
df['답안'].value_counts()

,count
답안,
3,2425
2,2314
1,2202
0,2053


In [ ]:
# 변환된 데이터를 train_df와 val_df로 나눔

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.shape, val_df.shape

((7195, 3), (1799, 3))

In [ ]:
# 이후 데이터셋을 생성하여 사용

class MCQDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        question = row['문제']
        choices = row['선택지']  # 이제 실제 리스트로 변환된 선택지를 사용
        options = " ".join([f"{i}. {choice}" for i, choice in enumerate(choices)])  # 선택지 번호 0부터 시작
        text = f"문제: {question} 선택지: {options}"

        # 정답 인덱스 (0부터 시작)
        label = int(row['답안'])

        # 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        encoding['labels'] = torch.tensor(label, dtype=torch.long)

        # 모든 텐서를 squeeze하여 반환
        return {key: val.squeeze(0) for key, val in encoding.items()}

In [ ]:
# Dataset 생성

train_dataset = MCQDataset(train_df, tokenizer, max_length=512)
val_dataset = MCQDataset(val_df, tokenizer, max_length=512)

In [ ]:
# 샘플 데이터의 텍스트를 디코딩하여 확인

for i in range(3):  # 상위 3개의 샘플을 확인
    sample = train_dataset[i]

    # input_ids를 텍스트로 변환
    input_ids = sample['input_ids']
    text = tokenizer.decode(input_ids, skip_special_tokens=True)

    # 출력
    print(f"Sample {i}:")
    print(f"Text: {text}")
    print(f"Label (정답): {sample['labels'].item()}")  # 텐서를 숫자로 변환하여 출력
    print("-" * 50)


Sample 0:
Text: 문제: 일반적으로 지진의 진앙을 고유하게 찾기 위해 최소 몇 개의 지진 관측소에서 S-P 도착 간격이 필요합니까? 선택지: 0. 1 1. 2 2. 3 3. 4
Label (정답): 2
--------------------------------------------------
Sample 1:
Text: 문제: 물체의 선형 크기를 3배로 확대하면 표면적은 _______ 증가합니다. 선택지: 0. 3배 그리고 부피는 9배. 1. 3배 그리고 부피는 27배. 2. 9배 그리고 부피는 27배. 3. 4배 그리고 부피는 8배.
Label (정답): 2
--------------------------------------------------
Sample 2:
Text: 문제: 프리마 페이시 의무의 개념을 설명하기 위해, Ross는 _______와(과) 비유를 듭니다. 선택지: 0. 자연 법칙. 1. 국가의 법률. 2. 독재자의 명령. 3. 우리의 감정.
Label (정답): 0
--------------------------------------------------


In [ ]:
# MCQDataset label 분포

import matplotlib.pyplot as plt
from collections import Counter

labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
Counter(labels)

Counter({2: 1865, 0: 1649, 1: 1753, 3: 1928})

객관식 추가 training

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # 검증 데이터셋 추가
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
    compute_metrics=compute_metrics,  # 정확도 계산 함수 추가
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,195 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.560000
2,2.505000
3,2.461300
4,2.250800
5,2.019400
6,1.854200
7,1.809800
8,2.203100
9,1.841700
10,2.264900


TrainOutput(global_step=60, training_loss=1.8028175195058187, metrics={'train_runtime': 116.5214, 'train_samples_per_second': 4.119, 'train_steps_per_second': 0.515, 'total_flos': 1.112830925340672e+16, 'train_loss': 1.8028175195058187, 'epoch': 0.06670372429127293})

In [ ]:
# 모델 평가

import random
import torch
from torch.utils.data import Subset

# 랜덤하게 100개의 샘플을 선택
random_indices = random.sample(range(len(val_dataset)), 10)

# Subset을 사용하여 작은 평가 데이터셋 생성
small_eval_dataset = Subset(val_dataset, random_indices)

# 평가 실행
eval_results = trainer.evaluate(eval_dataset=small_eval_dataset)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

# Test data 채우기

In [ ]:
test_data = "/content/test_data.csv"
df = pd.read_csv(test_data)

In [ ]:
# '선택지' 열을 리스트로 변환

df['선택지'] = df['선택지'].apply(ast.literal_eval)

MCQDataset 형태로 test_data 전처리 -> 제출

In [ ]:
import torch
from torch.utils.data import Dataset

class TestMCQDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        question = row['문제']
        choices = row['선택지']  # 이미 리스트로 변환된 선택지를 그대로 사용
        options = " ".join([f"{i}. {choice}" for i, choice in enumerate(choices)])
        text = f"문제: {question} 선택지: {options}"

         # 프롬프트 생성
        input_text = f"""
        문제: {question}
        선택지: {options}

        위의 선택지 중 하나를 선택하세요. 0, 1, 2, 3 중 하나의 번호로만 답변하세요.
        """

        # 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # 모든 텐서를 squeeze하여 반환
        return {key: val.squeeze(0) for key, val in encoding.items()}


In [ ]:
# TestMCQDataset 사용
test_dataset = TestMCQDataset(df, tokenizer, max_length=512)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 예측 수행
import re

model.eval()
predictions = []
failed_count = 0  # 예측 실패 값을 카운트하기 위한 변수

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: val.to("cuda") for key, val in batch.items()}
        outputs = model.generate(**inputs, max_new_tokens=10)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 예측값을 숫자로 변환
        predicted_answer = re.search(r'\b[0-3]\b', prediction)
        if predicted_answer:
            predicted_value = int(predicted_answer.group())
            predictions.append(predicted_value)
            print(f"예측된 값: {predicted_value}")  # 디버깅을 위한 출력
        else:
            predictions.append(0)  # 예측 실패 시 기본값으로 0을 추가
            failed_count += 1
            print("예측 실패: 0으로 처리")  # 디버깅을 위한 출력

예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 2
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 2
예측된 값: 0
예측된 값: 1
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 1
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예측된 값: 0
예

In [ ]:
submission_df = pd.DataFrame({
    'id': df['id'],
    'answer': predictions
})
submission_df.to_csv("submission(mcqdataset).csv", index=False)

In [ ]:
submission_df.head()
submission_df['answer'].value_counts()

,count
answer,
0,848
1,126
2,14
3,12


입력할 때 전처리 -> 제출

In [ ]:
from unsloth import FastLanguageModel

# 추론을 위한 최적화 설정
model = FastLanguageModel.for_inference(model)

import re

predictions = []
failed_predictions = 0

for idx, row in df.iterrows():
    # 문제와 선택지 구성
    question = row['문제']
    choices = row['선택지']  # 이미 리스트로 변환된 선택지를 그대로 사용
    options = " ".join([f"{i}. {choice}" for i, choice in enumerate(choices)])
    input_text = f"문제: {question} 선택지: {options}"

    # 토크나이징
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # 모델 예측
    outputs = model.generate(**inputs, max_new_tokens=10)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 예측값에서 숫자만 추출 (정규 표현식 사용)
    predicted_answer = re.search(r'\b[0-3]\b', prediction)

    if predicted_answer:
        predictions.append(int(predicted_answer.group()))  # 숫자로 변환
        print(f"모델 예측: {predicted_answer.group()}")  # 숫자만 출력
    else:
        print("예측 실패: 0으로 처리")
        predictions.append(0)  # 예측에 실패한 경우 기본값으로 0을 추가
        failed_predictions += 1


print(f"예측에 실패한 값의 개수: {failed_predictions}")

모델 예측: 0
모델 예측: 0
모델 예측: 1
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 1
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 1
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 1
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 0
모델 예측: 2


KeyboardInterrupt: 

In [ ]:
submission_df = pd.DataFrame({
    'id': df['id'],  # test_df의 id 열
    'answer': predictions  # 모델이 예측한 값
})

# CSV로 저장
submission_df.to_csv("submission.csv", index=False)